# Your first scraper
In this project, we will guide you step by step through the process of:

1. creating a self-contained development environment.
1. retrieving some information from an API (a website for computers)
2. leveraging it to scrape a website that does not provide an API
3. saving the output for later processing

Here we query an API for a list of countries and their past leaders. We then extract and sanitize their short bio from Wikipedia. Finally, we save the data to disk.

This task is often the first (coding) step of a datascience project and you will often come back to it in the future.

You will study topics such as *scraping*, *data structures*, *regular expressions*, *concurrency* and *file handling*. We will point out useful resources at the appropriate time. 

Let's dive in!

## Creating a clean environment

Use the [`venv`](https://docs.python.org/3/library/venv.html) command to create a new environment called `wikipedia_scraper_env`.

Activate it.

You will find more info about virtual environments in the course content and on the web.

## API Scraping

### A simple API query
You will start with the basics: how to do a simple request to an [API endpoint](../../2.python/2.python_advanced/05.Scraping/5.apis.ipynb).

You will use the [requests](https://requests.readthedocs.io/en/latest/) external library through the `import` keyword.

NOTE: external libraries need to be installed first. Check their documentation.

Check the [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/) section of the Requests documentation to:
1. use the `get()` method to connect to this endpoint: https://country-leaders.onrender.com/status
2. check if the `status_code` is equal to 200, which means OK.
    * if OK, `print()` the `text`` of the response.
    * if not, `print()` the `status_code`. 

Here is the signification of [HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).


In [ ]:
# import the requests library (1 line)
import requests

# assign the root url (without /status) to the root_url variable for ease of reference (1 line)
url = "https://country-leaders.onrender.com"

# assign the /status endpoint to another variable called status_url (1 line)
status_url = url + "/status"

# query the /status endpoint using the get() method and store it in the req variable (1 line)
req = requests.get(status_url)

# check the status_code using a condition and print appropriate messages (4 lines)
if req.status_code == 200:
    print(req.text)
else:
    print(f"Error: {req.status_code}")

### Dealing with JSON

[JSON](https://quickref.me/json) is the preferred format to deal with data over the web. You cannot avoid it so you would better get acquainted.

Connect to another endpoint called `/countries` but this time the API will return data in the JSON format. Check what to do in the Requests [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/).


In [ ]:
# Set the countries_url variable (1 line)
countries_url = url + "/countries"

# query the /countries endpoint using the get() method and store it in the req variable (1 line)
req = requests.get(countries_url)

# Get the JSON content and store it in the countries variable (1 line)
countries = req.json()

# display the request's status code and the countries variable (1 line)
print("{req.status_code}")
print(countries)


### Cookies anyone?

It looks like the access to this API is restricted...
Query the `/cookie` endpoint and extract the appropriate field to access your cookie.

You will need to use this cookie in each of the following API requests.

In [ ]:
# Set the cookie_url variable (1 line)
cookies_url = url + "/cookie"

# Query the enpoint, set the cookies variable and display it (2 lines)
cookies_req = requests.get(cookies_url)

cookies = cookies_req.cookies.get_dict()
cookies

Try to query the countries endpoint using the cookie, save the output and print it.

In [ ]:
# query the /countries endpoint, assign the output to the countries variable (1 line)
countries_req = requests.get(countries_url,  cookies=cookies)

# display the countries variable (1 line)
countries = countries_req.json()

print(countries)


Chances are the cookie has expired... Thanksfully, you got a nice error message. For now, simply execute the last 2 cells quickly so you get a result.

### Getting the actual data from the API

Query the `/leaders` endpoint.

In [ ]:
# Set the leaders_url variable (1 line)
leaders_url = url + "/leaders"

# query the /leaders endpoint, assign the output to the leaders variable (1 line)
leaders_req = requests.get(leaders_url, cookies=cookies)
leaders = leaders_req.json()

# display the leaders variable (1 line)
print(leaders)


It looks like this endpoint requires additional information in order to return its result. Check the API [*documentation*](https://country-leaders.onrender.com/docs) in your web browser.

Change the query to accept *parameters*. You should know where to find help by now.

In [ ]:
# query the /leaders endpoint using cookies and parameters (take any country in countries)
country = 'us'
leaders_req_params = requests.get(leaders_url, params={'country': country}, cookies=cookies)

# assign the output to the leaders variable (1 line)
leaders = leaders_req_params.json()

# display the leaders variable (1 line)
print(leaders)


### A sneak peak at the data (finally)

Look inside a few examples. Notice the dictionary keys available for each entry. You have your first example of *structured data*. This data was sanitized for your benefit, meaning it is readily exploitable without modification.

You will also notice there is a Wikipedia link for each entry. You will need to extract additional information there. This will be a case of *semi-structured* data.

The /countries endpoint returns a `list` of several country codes.

You need to loop through this list and query the /leaders endpoint for each one. Save each `json` result in a dictionary called `leaders_per_country`.

In [ ]:
# 4 lines
leaders_per_country = {}
for countrie in countries:
    leaders_req_params = requests.get(leaders_url, params={'country': countrie}, cookies=cookies)
    leaders = leaders_req_params.json()
    leaders_per_country[countrie] = leaders

print(leaders_per_country)




In [ ]:
# or 1 line


It is finally time to create a `get_leaders()` function for the above code. You will build on it later-on. This function takes no parameter. Inside it, you will need to:
1. define the urls
2. get the cookies
2. get the countries
3. loop over them and save their leaders in a dictionary
4. return the dictionary

In [ ]:
# < 15 lines
import requests

def get_leaders():
    url = "https://country-leaders.onrender.com"
    cookies_url = url + "/cookie"
    countries_url = url + "/countries"
    leaders_url = url + "/leaders"
    
    cookies_req = requests.get(cookies_url)
    cookies = cookies_req.cookies.get_dict()
    
    countries_req = requests.get(countries_url, cookies=cookies)
    countries = countries_req.json()

    leaders_per_country = {}
    
    for country in countries:
        leaders_req = requests.get(leaders_url, params={'country': country}, cookies=cookies)
        leaders = leaders_req.json()
        leaders_per_country[country] = leaders
    
    return leaders_per_country


Test your function, save the result in the `leaders_per_country` dictionary and check its ouput.

In [ ]:
# 2 lines
leaders_per_country = get_leaders()

print(leaders_per_country)

## Extracting data from Wikipedia

Query one of the leaders' Wikipedia urls and display its `text` (not JSON).

In [ ]:
for leader in leaders_per_country["us"]:
    print(leader['wikipedia_url'])

In [ ]:
# 3 lines
import requests
from bs4 import BeautifulSoup

content = requests.get('https://en.wikipedia.org/wiki/George_Washington')

soup = BeautifulSoup(content.text, 'html')

print(soup.get_text())



Ouch! You get the raw HTML code of the webpage. If you try to deal with it without tools, you will be there all night. Instead, use the [beautiful soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) *external* library. You will find more info about it [here](../../2.python/2.python_advanced/05.Scraping/1.beautifulsoup_basic.ipynb) and [here](../../2.python/2.python_advanced/05.Scraping/2.beautifulsoup_advanced.ipynb)

Using the Quickstart section, start by importing the library and loading the output of your `get_text()` function.

Use the `prettify()` function and print it to take a look. You will start the actual parsing in the next step.

In [ ]:
# 3 lines
import requests
from bs4 import BeautifulSoup

content = requests.get('https://en.wikipedia.org/wiki/George_Washington')

soup = BeautifulSoup(content.text, 'html')

pretty_content = soup.prettify()
print(pretty_content)


That looks better but you need to extract the right part of the webpage: the text of the first paragraph.

It is a bit tricky because Wikipedia pages slightly differ in structure from one language to the next. We cannot simply get the text for the first HTML paragraph.

You will start by getting all the HTML paragraphs from the HTML source and saving them in the `paragraphs` variable.

Use the documentation or google the appropriate keywords.

In [ ]:
# 2 lines
paragraphs = soup.find_all('p')

first_paragraph = paragraphs[1].get_text()
print(first_paragraph)

If you try different urls, you might find that the paragraph you want may be at a different index each time.

That is where you need to be clever and ask yourself what would be a reliable way to identify the right index ie. which string matches only the first paragraph whatever the language...

Spend a good 30 minutes on the problem and brainstorm with your fellow learners. If you come out empty handed, ask your coach.

1. Loop over the HTML paragraphs
2. When you have identified the correct one
  * store the [text](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#output) inside the `first_paragraph` variable
  * exit the loop

In [ ]:
# <10 lines

import re

for country_code, leaders in leaders_per_country.items():
    for leader in leaders:
        url = leader['wikipedia_url']
        content = requests.get(url)

        soup = BeautifulSoup(content.text, 'html')

        paragraphs = soup.find_all('p')

        for paragraph in paragraphs[1:]:
            paragraph_text = paragraph.get_text().strip()
            if re.search(r'\d', paragraph_text):
                first_paragraph = paragraph_text
                break

        print(f"({url}): {first_paragraph}")




At this stage, you can create a function to maintain consistency in your code. We will give you its *skeleton*, you will copy the code you wrote and make it work inside a function.

Don't forget to test your function.

In [ ]:
# 10 lines
# def get_first_paragraph(wikipedia_url):
#   print(wikipedia_url) # keep this for the rest of the notebook
#   [insert your code]
#   return first_paragraph

def get_first_paragraph(wikipedia_url):
    
    content = requests.get(wikipedia_url)
    soup = BeautifulSoup(content.text, 'html')
    paragraphs = soup.find_all('p')

    for paragraph in paragraphs[1:]:
        paragraph_text = paragraph.get_text()
        if re.search(r'\d', paragraph_text): 
            return paragraph_text
    



In [ ]:
# Test: 3 lines
for country_code, leaders in leaders_per_country.items():
    for leader in leaders:
        first_paragraph = get_first_paragraph(leader['wikipedia_url'])
        print(first_paragraph)

### Regular expressions to the rescue

Now that you have extracted the content of the first paragraph, the only thing that remains to finish your Wikipedia scraper is to sanitize the output.

Indeed some Wikipedia references, HTML code, phonetic pronunciation etc. may linger. You might find *regular expressions* handy to get rid of them and obtain pristine text. You will find some useful documentation about regular expressions [here](../../2.python/2.python_advanced/03.Regex/regex.ipynb)

Once you have one of your regex working online, try it in the cell below. 

Hints: 
* Check the `sub()` method documentation.
* Make sure to test urls in different languages. Some may look good but other do not.

In [ ]:
# 3 lines
import re

def clean_text(text):
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'/[^;]*;', '', text)
    text = re.sub(r'[ⓘ]', '', text)
    text = re.sub(r'\s{2,}', ' ', text)
    text = re.sub(r'\(/\s*[^)]*\)', '', text)
    text = re.sub(r'\(\s+', '(', text)

    text = text.strip()
    
    return text

for country_code, leaders in leaders_per_country.items():
    for leader in leaders:
        first_paragraph = get_first_paragraph(leader['wikipedia_url'])
        print(clean_text(first_paragraph))

Overwrite the `get_first_paragraph()` function by applying your regex to the first paragraph before returning it.

In [ ]:
# 10 lines
def get_first_paragraph(wikipedia_url):
    
    content = requests.get(wikipedia_url)
    soup = BeautifulSoup(content.text, 'html')
    paragraphs = soup.find_all('p')

    for paragraph in paragraphs[1:]:
        paragraph_text = paragraph.get_text()
        if re.search(r'\d', paragraph_text):
            return clean_text(paragraph_text)
    

for country_code, leaders in leaders_per_country.items():
    for leader in leaders:
        first_paragraph = get_first_paragraph(leader['wikipedia_url'])
        print(clean_text(first_paragraph))

Come up with other regexes to capture other patterns and sanitize the outputs completely. Modify your `get_first_paragraph()` function accordingly.

In [ ]:
# < 20 lines
def get_first_paragraph(wikipedia_url):
    
    content = requests.get(wikipedia_url)
    soup = BeautifulSoup(content.text, 'html')
    paragraphs = soup.find_all('p')

    for paragraph in paragraphs[1:]:
        paragraph_text = paragraph.get_text()
        if re.search(r'\d', paragraph_text):
            return clean_text(paragraph_text)

for country_code, leaders in leaders_per_country.items():
    for leader in leaders:
        first_paragraph = get_first_paragraph(leader['wikipedia_url'])
        print(clean_text(first_paragraph))

## Putting it all together

Let's go back to your `get_leaders()` function and update it with an *inner* loop over each leader. You will query the url provided and extract the first paragraph using the `get_first_paragraph()` function you just finished. You will then update that `leader`'s dictionary and move on to the next one.

Notice, the rest of the code should not change since you modify the leader's data one by one.

In [ ]:
# < 20 lines

def get_leaders():
    url = "https://country-leaders.onrender.com"
    cookies_url = url + "/cookie"
    countries_url = url + "/countries"
    leaders_url = url + "/leaders"
    
    cookies_req = requests.get(cookies_url)
    cookies = cookies_req.cookies.get_dict()
    
    countries_req = requests.get(countries_url, cookies=cookies)
    countries = countries_req.json()

    leaders_per_country = {}
    
    for country in countries:
        leaders_req = requests.get(leaders_url, params={'country': country}, cookies=cookies)
        leaders = leaders_req.json()
        leaders_per_country[country] = leaders
    
    leaders_dict = {}

    for country_code, leaders in leaders_per_country.items():
        for leader in leaders:
          leader_name = f"{leader['first_name']} {leader['last_name']}"
          first_paragraph = get_first_paragraph(leader['wikipedia_url'])
          leaders_dict[leader_name] = first_paragraph
    return leaders_dict




In [ ]:
# Check the output of your function (2 lines)
get_leaders()

Does the function crash in the middle of the loop? Chances are the cookies have expired while looping over the leaders.

Modify your function with an *exception* or check if the `status_code` is a cookie error. In either case, get new cookies and query the api again.

If your code did not crash,

In [ ]:
# < 25 lines
def get_first_paragraph(wikipedia_url):
    
    content = requests.get(wikipedia_url)
    soup = BeautifulSoup(content.text, 'html')
    paragraphs = soup.find_all('p')

    for paragraph in paragraphs[1:]:
        paragraph_text = paragraph.get_text()
        if re.search(r'\d', paragraph_text):
            return clean_text(paragraph_text)

def get_leaders():
    url = "https://country-leaders.onrender.com"
    cookies_url = url + "/cookie"
    countries_url = url + "/countries"
    leaders_url = url + "/leaders"
    
    cookies_req = requests.get(cookies_url)
    cookies = cookies_req.cookies.get_dict()
    
    countries_req = requests.get(countries_url, cookies=cookies)
    countries = countries_req.json()

    leaders_per_country = {}
    
    for country in countries:
        leaders_req = requests.get(leaders_url, params={'country': country}, cookies=cookies)
        leaders = leaders_req.json()
        leaders_per_country[country] = leaders
    
    leaders_dict = {}

    for country_code, leaders in leaders_per_country.items():
        for leader in leaders:
          leader_name = f"{leader['first_name']} {leader['last_name']}"
          first_paragraph = get_first_paragraph(leader['wikipedia_url'])
          leaders_dict[leader_name] = first_paragraph
    return leaders_dict


get_leaders()

Check the output of your function again.

In [ ]:
# Check the output of your function (1 line)


Well done! It took a while however... Let's speed things up. The main *bottleneck* is the loop. We call on the Wikipedia website many times.

You will use the same *session* to call all the wikipedia pages. Check the *Advanced Usage* section of the Requests module's documentation.

Start by modifying the `get_first_paragraph()` function to accept a session parameter and adjust the `get()` method call.

In [ ]:
# < 20 lines

def get_first_paragraph(wikipedia_url, session):
    
    content = session.get(wikipedia_url)
    soup = BeautifulSoup(content.text, 'html')
    paragraphs = soup.find_all('p')

    for paragraph in paragraphs[1:]:
        paragraph_text = paragraph.get_text()
        if re.search(r'\d', paragraph_text):
            return clean_text(paragraph_text)


def get_leaders():
    url = "https://country-leaders.onrender.com"
    cookies_url = url + "/cookie"
    countries_url = url + "/countries"
    leaders_url = url + "/leaders"
    
    cookies_req = requests.get(cookies_url)
    cookies = cookies_req.cookies.get_dict()
    
    countries_req = requests.get(countries_url, cookies=cookies)
    countries = countries_req.json()

    leaders_per_country = {}
    
    with requests.Session() as session:
        for country in countries:
            leaders_req = session.get(leaders_url, params={'country': country}, cookies=cookies)
            leaders = leaders_req.json()
            leaders_per_country[country] = leaders

        leaders_dict = {}
        for country_code, leaders in leaders_per_country.items():
            for leader in leaders:
                leader_name = f"{leader['first_name']} {leader['last_name']}"
                first_paragraph = get_first_paragraph(leader['wikipedia_url'], session)
                leaders_dict[leader_name] = first_paragraph

    return leaders_dict


get_leaders()


Modify your `get_leaders()` function to make use of a single session for all the Wikipedia calls.
1. create a `Session` object outside of the loop over countries.
2. pass it to the `get_first_paragraph()` function as an argument.

In [ ]:
# <25 lines
print(leaders_dict)

Test your new functions.



In [58]:
import requests
from bs4 import BeautifulSoup
import re
import json


def clean_text(text):
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'/[^;]*;', '', text)
    text = re.sub(r'[ⓘ]', '', text)
    text = re.sub(r'\s{2,}', ' ', text)
    text = re.sub(r'\(/\s*[^)]*\)', '', text)
    text = re.sub(r'\(\s+', '(', text)

    text = text.strip()
    
    return text

def save(dict_as_json, file_name):
    with open(file_name, 'w') as f:
        json.dump(dict_as_json, f)

def load(file_name):
    with open(file_name, 'r') as f:
        return json.load(f)


def get_first_paragraph(wikipedia_url, session):
    
    content = session.get(wikipedia_url)
    soup = BeautifulSoup(content.text, 'html.parser')
    paragraphs = soup.find_all('p')

    for paragraph in paragraphs[1:]:
        paragraph_text = paragraph.get_text()
        if re.search(r'\d', paragraph_text):
            return clean_text(paragraph_text)


def get_leaders():
    url = "https://country-leaders.onrender.com"
    cookies_url = url + "/cookie"
    countries_url = url + "/countries"
    leaders_url = url + "/leaders"
    
    cookies_req = requests.get(cookies_url)
    cookies = cookies_req.cookies.get_dict()
    
    countries_req = requests.get(countries_url, cookies=cookies)
    countries = countries_req.json()

    leaders_per_country = {}
    
    with requests.Session() as session:
        for country in countries:
            leaders_req = session.get(leaders_url, params={'country': country}, cookies=cookies)
            leaders = leaders_req.json()
            leaders_per_country[country] = leaders

        leaders_dict = {}
        for country_code, leaders in leaders_per_country.items():
            for leader in leaders:
                leader_name = f"{leader['first_name']} {leader['last_name']}"
                first_paragraph = get_first_paragraph(leader['wikipedia_url'], session)
                leaders_dict[leader_name] = first_paragraph

    return leaders_dict

get_leaders()

{'George Washington': 'George Washington (February 22, 1732\xa0– December 14, 1799) was a Founding Father of the United States, military officer, and farmer who served as the first president of the United States from 1789 to 1797. Appointed by the Second Continental Congress as commander of the Continental Army in 1775, Washington led Patriot forces to victory in the American Revolutionary War and then served as president of the Constitutional Convention in 1787, which drafted the current Constitution of the United States. Washington has thus become commonly known as the "Father of his Country".',
 'Barack Obama': 'Barack Hussein Obama II (born August 4, 1961) is an American politician who served as the 44th president of the United States from 2009 to 2017. As a member of the Democratic Party, he was the first African-American president in U.S. history. Obama previously served as a U.S. senator representing Illinois from 2005 to 2008 and as an Illinois state senator from 1997 to 2004.'

## Saving your hard work

The final step is to save the ``leaders_per_country`` dictionary in the `leaders.json` file using the [json](https://docs.python.org/3/library/json.html) module. Check out the `with` statement.

In [ ]:
# 3 lines
import json

with open('leaders.json', 'w') as f:
        json.dump(leaders_dict, f)


Make sure the file can be read back. Write the code to read the file. And check the variables are the same.

In [ ]:
# 3 lines
with open('leaders.json', 'r') as f:
    print(json.load(f))


Make a function `save(leaders_per_country)` to call this code easily.

In [ ]:
# 3 lines
# 3 lines

def save(dict_as_json, file_name):
    with open(file_name, 'w') as f:
        json.dump(dict_as_json, f)

def load(file_name):
    with open(file_name, 'r') as f:
        return json.load(f)
    
save(leaders_dict, "Leaders.json")
load("Leaders.json")


In [ ]:
# Call the function (1 line)


## Tidy things up in a stand-alone python script

Congratulations! You now have a working scraper! However, your code is scattered throughout this notebook along side the tutorials. Hardly production ready...

Copy and paste what you need in a separate `leaders_scraper.py` file.
Make sure it works by calling `python3 leaders_scraper.py`

## To go further

If you want to practice scraping, you can read this section and tackle the exercise.

You have noticed the API returns very partial results for country leaders. Many are missing. Overwrite the `get_leaders()` function to get its list from Wikipedia and extract their *personal details* from the frame on the side.

Good luck!